# City information collection

In this document we will run and install libraries sqlalchemy, pymysql, pandas, BeaufifulSoup and requests,.

The goal is to scape city information from Wikipedia, move it from JSON into a dataframe and push it to MYSQL

In [3]:
!pip install sqlalchemy
!pip install pymysql
import pandas as pd
import requests
from bs4 import BeautifulSoup


Our challenge collect infomation on the cities Berlin, Hamburg, and Munich

In [12]:
# set-up scraping infrastructure

# using a dictionary for easy transposition to Dataframe later
cities_information = {'city':['Berlin','Munich','Hamburg'],'latitude':[],'longitude':[],'country':[]}
headers = {'User-Agent': 'Chrome/134.0.0.0'} # needed to act as a fake person

In [13]:
# loop code

for city_name in cities_information['city']: # iterate through the first list in the dictionary
    url = f'https://en.wikipedia.org/wiki/{city_name}' # main part of the URL plus the city name given
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Use try/except to avoid crashes if a tag is missing
    try:
        lat = soup.find( class_='latitude').get_text() # getting lat and if not working it updates with a null value. The class factor is how we find the text
    except AttributeError:
        lat = None

    try:
        lon = soup.find(class_='longitude').get_text()
    except AttributeError:
        lon = None

    try:
        # This gets the first infobox data cell that contains a country
        country = soup.find('td', class_='infobox-data').get_text()
    except AttributeError:
        country = None

 # Append data back into the dictionary
    cities_information['latitude'].append(lat)
    cities_information['longitude'].append(lon)
    cities_information['country'].append(country)

cities_information

{'city': ['Berlin', 'Munich', 'Hamburg'],
 'latitude': ['52°31′12″N', '48°08′15″N', '53°33′N'],
 'longitude': ['13°24′18″E', '11°34′30″E', '10°00′E'],
 'country': ['Germany', 'Germany', 'Germany']}

In [14]:
cities_information

{'city': ['Berlin', 'Munich', 'Hamburg'],
 'latitude': ['52°31′12″N', '48°08′15″N', '53°33′N'],
 'longitude': ['13°24′18″E', '11°34′30″E', '10°00′E'],
 'country': ['Germany', 'Germany', 'Germany']}

In [15]:
# creation of a function to add a city

def cities_info_search(city_name):

    cities_information['city'].append(city_name)

    url = f'https://en.wikipedia.org/wiki/{city_name}'
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

     # Use try/except to avoid crashes if a tag is missing
    try:
        lat = soup.find( class_='latitude').get_text()
    except AttributeError:
        lat = None

    try:
        lon = soup.find(class_='longitude').get_text()
    except AttributeError:
        lon = None

    try:
        # This gets the first infobox data cell that contains a country
        country = soup.find('td', class_='infobox-data').get_text()
    except AttributeError:
        country = None

    cities_information['latitude'].append(lat)
    cities_information['longitude'].append(lon)
    cities_information['country'].append(country)

    return print(cities_information)

In [18]:
cities_info_search('Freiburg') # also added Paris, Brussels

{'city': ['Berlin', 'Munich', 'Hamburg', 'Freiburg', 'Paris', 'Brussels'], 'latitude': ['52°31′12″N', '48°08′15″N', '53°33′N', '47°59′42″N', '48°51′24″N', '50°50′48″N'], 'longitude': ['13°24′18″E', '11°34′30″E', '10°00′E', '07°51′00″E', '2°21′8″E', '04°21′09″E'], 'country': ['Germany', 'Germany', 'Germany', 'Germany', 'France', 'Belgium']}


In [20]:
# turn into a dataframe
major_cities_df = pd.DataFrame(data = cities_information)
major_cities_df

,city,latitude,longitude,country
0,Berlin,52°31′12″N,13°24′18″E,Germany
1,Munich,48°08′15″N,11°34′30″E,Germany
2,Hamburg,53°33′N,10°00′E,Germany
3,Freiburg,47°59′42″N,07°51′00″E,Germany
4,Paris,48°51′24″N,2°21′8″E,France
5,Brussels,50°50′48″N,04°21′09″E,Belgium


## Send to MYSQL

Code used in MySQL to create a new scheme or relational database to store the information we will collect for the rest of the project


schema = "Europeancities"
host = "127.0.0.1"
user = "root"
password = "YOUR PASSWORD HERE"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [22]:
# We will have two dataframes city_name and city_information

cities_unique = major_cities_df["city"].unique()

city_name = pd.DataFrame({"city": cities_unique})

city_name

,city
0,Berlin
1,Munich
2,Hamburg
3,Freiburg
4,Paris
5,Brussels


MySQL Table Creation code

USE Europeancities;

CREATE TABLE city_name (
    city_name_id INT AUTO INCREMENT, -- to automatically set an id for primary key
    city VARCHAR (100) -- various characters
    PRIMARY KEY (city_name_id)
);


In [ ]:
cities_df.to_sql('city_name',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [22]:
cities_from_sql = pd.read_sql("city_name", con=connection_string) # retriving information from SQL
cities_from_sql

In [ ]:
# merge to make sure both have the primary key to be used later as a foreign key
city_info_df = major_cities_df.merge(cities_from_sql,
                                   on = "city",
                                   how="left")

city_info_df

In [ ]:
city_info_df = city_info_df.drop(columns=["city"])

city_info_df

MySQL code:

CREATE TABLE (
    city_info_id INT AUTO INCREMENT NOT NULL,
    latitude	FLOAT  NOT NULL,
	longitude FLOAT  NOT NULL,
    country VARCHAR (100) NOT NULL,
    PRIMARY KEY (city_info_id), -- primary key to id each airport
	FOREIGN KEY (city_name_id) REFERENCES city_name(city_name_id) --
)

In [ ]:
city_info_df.to_sql('city_info',
                if_exists='append',
                con=connection_string,
                index=False)

Note that not changing the latitude into decimals before will cause headaches and will be addressed in the next few documents
